In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-21"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1413,2024-06-21,Mundo Amistoso Internacional,10:30,Bélgica U20,Alemanha U20,3.31,1.30,141.5,1.85,1.85,0.0,0.00,0.00,Yes8IU9E,0.302115,0.769231,0.540541,0.540541,0.071346,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,73,66,1.56,2.72,0.000,0.000,0.616609,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1414,2024-06-21,Mundo Amistoso Internacional,13:00,Turquia U20,Grécia U20,1.41,2.76,143.5,1.85,1.85,0.0,0.00,0.00,ve9u0Mgm,0.709220,0.362319,0.540541,0.540541,0.071539,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,94,82,1.42,1.06,0.000,0.000,0.457839,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1415,2024-06-21,Mundo Amistosos Internacionais,14:00,Portugal U20 F,Polónia U20 F,1.39,2.84,125.5,1.83,1.87,0.0,0.00,0.00,norrzATQ,0.719424,0.352113,0.546448,0.534759,0.071537,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.484778,0.015289,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1416,2024-06-21,Mundo Amistosos Internacionais,16:15,Republica Checa U20 F,Espanha U20 F,8.25,1.05,135.5,1.85,1.85,0.0,0.00,0.00,rVrDUmil,0.121212,0.952381,0.540541,0.540541,0.073593,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,1.094875,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1417,2024-06-21,Nova Zelândia Nbl,04:30,Southland Sharks,Whai,2.31,1.57,162.5,1.85,1.85,0.0,0.00,0.00,CxeAGvMb,0.432900,0.636943,0.540541,0.540541,0.069843,493.616,271.846738,0.550725,0.6,1.341641,2.236068,707.60,6.760,4.520271,0.668679,-62.0,383.662,259.178376,0.675538,1.2,1.643168,1.369306,758.45,5.196,3.404244,0.655166,-45.0,61,77,11.60,9.85,493.616,0.000,0.269721,0.000000,NaN,-2.41,-0.482,-2.717842,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1418,2024-06-21,Paraguai Lnb,20:30,Dep. San Jose,Colonias Gold,1.42,2.72,150.5,1.85,1.85,0.0,0.00,0.00,6PR3um9c,0.704225,0.367647,0.540541,0.540541,0.071872,114.212,45.394146,0.397455,2.4,1.341641,0.559017,101.25,1.464,0.591718,0.404179,22.0,189.684,57.254323,0.301841,1.8,1.643168,0.912871,231.04,2.604,0.904367,0.347299,7.0,75,64,1.35,3.61,106.842,141.994,0.444077,0.000000,NaN,1.21,0.242,1.735537,0.000000,0.0,0.000000,1.05,0.210,8.190476,0.000000,0.0,0.000000
1419,2024-06-21,Paraguai Lnb,20:30,Felix Perez Cardozo,Olimpia,2.61,1.45,148.5,1.84,1.84,0.0,0.00,0.00,EkVBwRwA,0.383142,0.689655,0.543478,0.543478,0.072797,751.036,956.474618,1.273540,1.2,1.643168,1.369306,202.34,9.258,10.714034,1.157273,1.0,550.970,1006.961630,1.827616,1.8,1.643168,0.912871,76.86,6.664,11.927989,1.789914,53.0,67,61,3.02,1.26,611.544,563.530,0.404061,0.000000,NaN,27.46,5.492,0.293154,0.000000,0.0,0.000000,26.96,5.392,0.083457,0.000000,0.0,0.000000
1420,2024-06-21,Venezuela Superliga,19:30,Frontinos,Broncos

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
